# CHARM Data Preprocessing Statistical Analysis

This notebook analyzes CHARM experimental data preprocessing results including:
- Raw data quality statistics
- Data cleaning effectiveness assessment
- RNA and DNA reads distribution analysis
- Paired data (pairs) quality evaluation
- Multi-omics data integration analysis

## 1. Environment Setup and Library Loading

In [6]:
suppressPackageStartupMessages({
  library(tidyverse)
  library(ggpubr)
  library(yaml)
  library(patchwork)
})

options(repr.matrix.max.cols = 100, repr.matrix.max.rows = 20)

## 2. Configuration and Data Path Setup

In [7]:
config <- read_yaml(file = "../CHARM_preprocess_pipeline/config.yaml")

stat_dir <- "../stat/"
result_dir <- "../result/"

## 3. Raw Data Statistics Loading

Load various statistical files including raw reads, DNA reads, RNA reads and other basic statistics.

In [8]:
# Convert reads count to Gb and extract cell names
Raw <- read_table(paste0(stat_dir, "raw.fq.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X2 = X2 / 4 * 300 / 1000000000,
    X1 = strsplit(X1, split = "/")[[1]][3],
    X1 = strsplit(X1, split = "_")[[1]][1]
  )

DNA <- read_table(paste0(stat_dir, "dna.fq.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X2 = X2 / 4 * 300 / 1000000000,
    X1 = strsplit(X1, split = "/")[[1]][3]
  )

RNA <- read_table(paste0(stat_dir, "rna.fq.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X2 = X2 / 4 * 300 / 1000000000,
    X1 = strsplit(X1, split = "/")[[1]][3]
  )


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)



## 4. Paired Data (Pairs) Statistics Loading

Load Hi-C paired data statistics from various processing stages.

In [9]:
raw_pairs <- read_table(paste0(stat_dir, "raw.pairs.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(X1 = strsplit(X1, split = "/")[[1]][3])

pairs_dedup <- read_table(paste0(stat_dir, "pairs.dedup.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(X1 = strsplit(X1, split = "/")[[1]][3])

# Cleaned paired data statistics
pairs_c1 <- read_table(paste0(stat_dir, "pairs.c1.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X1 = strsplit(X1, split = "/")[[1]][5],
    X1 = str_replace(X1, ".pairs.gz", "")
  )

pairs_c12 <- read_table(paste0(stat_dir, "pairs.c12.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X1 = strsplit(X1, split = "/")[[1]][5],
    X1 = str_replace(X1, ".pairs.gz", "")
  )

pairs_c123 <- read_table(paste0(stat_dir, "pairs.c123.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X1 = strsplit(X1, split = "/")[[1]][5],
    X1 = str_replace(X1, ".pairs.gz", "")
  )

# Inter-chromosomal pairs statistics
inter_pairs_c123 <- read_table(paste0(stat_dir, "inter.pairs.c123.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(
    X1 = strsplit(X1, split = "/")[[1]][5],
    X1 = str_replace(X1, ".pairs.gz", "")
  )


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)



## 5. RNA Expression Data Statistics

Load RNA-seq data statistics at gene and exon levels.

In [27]:
# Y/X chromosome ratio statistics
yperx <- read_table(paste0(stat_dir, "yperx.stat"), col_names = FALSE) %>%
  arrange(X1) %>%
  rowwise() %>%
  mutate(X1 = strsplit(X1, split = "/")[[1]][2])

# Gene-level expression statistics
RNAres <- read_table(paste0(result_dir, "RNA_Res/counts.gene.total.format.tsv"))
featureStat <- cbind(
  as.data.frame(colSums(RNAres %>% dplyr::select(-gene))),
  colSums(RNAres %>% dplyr::select(-gene) != 0)
) %>%
  rownames_to_column("X1")

# Exon-level expression statistics
RNAresExon <- read_table(paste0(result_dir, "RNA_Res/counts.exon.total.format.tsv"))
featureStatExon <- cbind(
  as.data.frame(colSums(RNAresExon %>% dplyr::select(-gene))),
  colSums(RNAresExon %>% dplyr::select(-gene) != 0)
) %>%
  rownames_to_column("X1")


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)




-- Column specification --------------------------------------------------------
cols(
  gene = col_character(),
  R1P10013 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  gene = col_character(),
  R1P10013 = col_double()
)



## 6. Data Integration and Statistical Table Construction

Integrate all statistical information into a single data frame based on configuration settings.

In [28]:
# Basic statistical data integration
stat <- Raw %>%
  left_join(DNA, by = "X1") %>%
  left_join(RNA, by = "X1") %>%
  left_join(yperx, by = "X1") %>%
  left_join(raw_pairs, by = "X1") %>%
  left_join(pairs_dedup, by = "X1") %>%
  left_join(pairs_c1, by = "X1") %>%
  left_join(pairs_c12, by = "X1") %>%
  left_join(pairs_c123, by = "X1") %>%
  left_join(inter_pairs_c123, by = "X1")

# Add RNA SNP separation statistics based on configuration
if (config$if_RNA_snp_split) {
  RNAresGenome1 <- read_table(paste0(result_dir, "RNA_Res/counts.gene.genome1.tsv"))
  RNAresGenome2 <- read_table(paste0(result_dir, "RNA_Res/counts.gene.genome2.tsv"))
  
  featureStatGenome1 <- cbind(
    as.data.frame(colSums(RNAresGenome1 %>% select(-gene))),
    colSums(RNAresGenome1 %>% select(-gene) != 0)
  ) %>%
    rownames_to_column("X1")
  
  featureStatGenome2 <- cbind(
    as.data.frame(colSums(RNAresGenome2 %>% select(-gene))),
    colSums(RNAresGenome2 %>% select(-gene) != 0)
  ) %>%
    rownames_to_column("X1")
  
  stat <- stat %>%
    left_join(featureStat, by = "X1") %>%
    left_join(featureStatExon, by = "X1") %>%
    left_join(featureStatGenome1, by = "X1") %>%
    left_join(featureStatGenome2, by = "X1")
  
  names(stat) <- c(
    "cellname", "Rawreads", "DNAreads", "RNAreads", "yperx",
    "raw_pairs", "pairs_dedup", "pairs_clean1", "pairs_clean2", "pairs_clean3",
    "inter_pairs_clean3", "UMIs_gene", "genes_gene", "UMIs_exon", "genes_exon",
    "UMIs_gene_genome1", "genes_gene_genome1", "UMIs_gene_genome2", "genes_gene_genome2"
  )
} else {
  stat <- stat %>%
    left_join(featureStat, by = "X1") %>%
    left_join(featureStatExon, by = "X1")
  
  names(stat) <- c(
    "cellname", "Rawreads", "DNAreads", "RNAreads", "yperx",
    "raw_pairs", "pairs_dedup", "pairs_clean1", "pairs_clean2", "pairs_clean3",
    "inter_pairs_clean3", "UMIs_gene", "genes_gene", "UMIs_exon", "genes_exon"
  )
}

## 7. Optional Statistical Information Addition

Add structural information and CHARM-specific statistics based on configuration settings.

In [31]:
# Add structural information if enabled
if (config$if_structure) {
  rmsd <- read_table(paste0(stat_dir, "rmsd.info"), col_names = FALSE) %>%
    rowwise() %>%
    mutate(
      X1 = str_split(X1, pattern = fixed("/")),
      cellname = X1[1],
      res = str_split(X1[3], fixed("."))[[1]][2],
      TOP3_RMSD = X5
    ) %>%
    dplyr::select(cellname, res, TOP3_RMSD) %>%
    spread(res, TOP3_RMSD) %>%
    arrange(cellname)
  
  stat <- stat %>% left_join(rmsd, by = "cellname")
}

# Add CHARM-specific statistics if enabled
if (config$if_charm) {
  reads <- tibble("cellname" = NA)
  
  for (i in 1:length(names(config$split))) {
    temp <- read_csv(
      paste0(stat_dir, names(config$split)[i], ".read.stat"),
      col_names = FALSE
    ) %>%
      rowwise() %>%
      mutate(X2 = X2 * 300 / 1000000000)
    
    names(temp) <- c("cellname", paste0(names(config$split)[i], "_reads"))
    reads <- reads %>% full_join(temp, by = "cellname")
  }
  
  reads <- reads %>% dplyr::filter(!is.na(cellname))
  stat <- stat %>% full_join(reads, by = "cellname")
}


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_character(),
  X3 = col_character(),
  X4 = col_character(),
  X5 = col_double()
)

Rows: 1 Columns: 2


-- Column specification --------------------------------------------------------
Delimiter: ","
chr (1): X1
dbl (1): X2

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (1): X1
dbl (1): X2

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


## 8. Derived Metrics Calculation

Calculate important quality control metrics and ratios.

In [32]:
plot_data <- stat %>%
  mutate(
    RNAreadsRatio = RNAreads / (RNAreads + DNAreads),
    pairsPerRead = raw_pairs / DNAreads / 1000000000 * 300,
    pairsValidRatio = pairs_clean3 / raw_pairs,
    interPairsRatio = inter_pairs_clean3 / pairs_clean3
  )

plot_data[is.na(plot_data)] <- 0

## 9. Data Overview

Display the integrated statistical data table.

In [34]:
print("=== CHARM Data Preprocessing Statistics Overview ===")
plot_data

[1] "=== CHARM Data Preprocessing Statistics Overview ==="


cellname,Rawreads,DNAreads,RNAreads,yperx,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,pairs_clean3,inter_pairs_clean3,UMIs_gene,genes_gene,UMIs_exon,genes_exon,1m.x,200k.x,20k.x,50k.x,1m.y,200k.y,20k.y,50k.y,ct_reads,atac_reads,RNAreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
R1P10013,3.753487,3.752322,0.0010437,0.005022,3736619,779808,768346,508560,508059,27553,2152,1154,299,219,4.232651,8.206307,20.8746,14.853,4.232651,8.206307,20.8746,14.853,0.0063276,0.007428,0.0002780704,0.2987445,0.1359676,0.05423189


---

# CHARM Experiment-Specific Analysis

**Note: The following sections are only executed for CHARM experiments**

## 10. CHARM Multi-omics Data Integration

If this is a CHARM experiment, run the following code for multi-omics data integration analysis.

In [ ]:
cellnames <- plot_data$cellname
print(paste("Detected", length(cellnames), "cells"))
print("Cell list:")
print(cellnames)

## 11. Seurat Object Creation and Multi-omics Integration

Use Signac and Seurat packages for CHARM data multi-omics integration analysis.

In [ ]:
suppressPackageStartupMessages({
  library(Signac)
  library(Seurat)
  library(EnsDb.Mmusculus.v79)
  library(BSgenome.Mmusculus.UCSC.mm10)
  library(future)
})

plan("multicore", workers = 10)

# Create ATAC-seq fragment object
tryCatch({
  atac_fragments <- CreateFragmentObject(
    path = "../result/fragments/atac.fragments.bgz",
    cells = cellnames
  )
  
  print("ATAC-seq fragment object created successfully")
}, error = function(e) {
  print(paste("ATAC-seq fragment object creation failed:", e$message))
  print("Please check if fragments file exists and is properly formatted")
})

## 12. Data Quality Assessment and Visualization

Generate quality control plots and statistical reports.

In [ ]:
if (nrow(plot_data) > 0) {
  # RNA/DNA reads ratio plot
  p1 <- ggplot(plot_data, aes(x = cellname, y = RNAreadsRatio)) +
    geom_bar(stat = "identity", fill = "steelblue") +
    labs(
      title = "RNA Reads Ratio",
      x = "Cell",
      y = "RNA/(RNA+DNA) Reads Ratio"
    ) +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
  # Valid pairs ratio plot
  p2 <- ggplot(plot_data, aes(x = cellname, y = pairsValidRatio)) +
    geom_bar(stat = "identity", fill = "coral") +
    labs(
      title = "Valid Pairs Ratio",
      x = "Cell",
      y = "Valid Pairs/Raw Pairs Ratio"
    ) +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
  combined_plot <- p1 / p2
  print(combined_plot)
} else {
  print("No data available for visualization")
}

## 13. Summary Report

Generate a comprehensive summary report of data preprocessing.

In [ ]:
cat("\n=== CHARM Data Preprocessing Summary Report ===\n")
cat(paste("Number of processed cells:", nrow(plot_data), "\n"))

if (nrow(plot_data) > 0) {
  cat("\n--- Data Quality Statistics ---\n")
  cat(paste("Average raw reads (Gb):", round(mean(plot_data$Rawreads, na.rm = TRUE), 3), "\n"))
  cat(paste("Average DNA reads (Gb):", round(mean(plot_data$DNAreads, na.rm = TRUE), 3), "\n"))
  cat(paste("Average RNA reads (Gb):", round(mean(plot_data$RNAreads, na.rm = TRUE), 3), "\n"))
  cat(paste("Average valid pairs:", round(mean(plot_data$pairs_clean3, na.rm = TRUE), 0), "\n"))
  cat(paste("Average detected genes:", round(mean(plot_data$genes_gene, na.rm = TRUE), 0), "\n"))
  cat(paste("Average UMI count:", round(mean(plot_data$UMIs_gene, na.rm = TRUE), 0), "\n"))
  
  cat("\n--- Quality Control Metrics ---\n")
  cat(paste("Average RNA reads ratio:", round(mean(plot_data$RNAreadsRatio, na.rm = TRUE), 4), "\n"))
  cat(paste("Average valid pairs ratio:", round(mean(plot_data$pairsValidRatio, na.rm = TRUE), 4), "\n"))
  cat(paste("Average inter-chromosomal pairs ratio:", round(mean(plot_data$interPairsRatio, na.rm = TRUE), 4), "\n"))
} else {
  cat("Warning: No valid data detected\n")
}

cat("\n=== Analysis Complete ===\n")